<a href="https://colab.research.google.com/github/ssingh1187/Unit-2-Kaggle-Competition/blob/master/Kaggle_Project_(Final_draft).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
test_features = pd.read_csv('https://raw.githubusercontent.com/ssingh1187/DS-Unit-2-Sprint-4-Model-Validation/master/test_features.csv')
train_features = pd.read_csv('https://raw.githubusercontent.com/ssingh1187/DS-Unit-2-Sprint-4-Model-Validation/master/train_features.csv')
train_labels = pd.read_csv('https://raw.githubusercontent.com/ssingh1187/DS-Unit-2-Sprint-4-Model-Validation/master/train_labels.csv')
sample_submission = pd.read_csv('https://raw.githubusercontent.com/ssingh1187/DS-Unit-2-Sprint-4-Model-Validation/master/train_labels.csv')


In [0]:
print("\ntrain_features null counts:")
print(train_features.isnull().sum())
print(train_labels.status_group.value_counts())
print("\ntest_features null counts:")
print(test_features.isnull().sum())


train_features null counts:
id                           0
amount_tsh                   0
date_recorded                0
funder                    3635
gps_height                   0
installer                 3655
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 371
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            3334
recorded_by                  0
scheme_management         3877
scheme_name              28166
permit                    3056
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_qual

In [0]:
# Ignoring null columns
train_features.dropna(axis=1, inplace=True)
test_features.dropna(axis=1, inplace=True)

In [0]:
# creating age features

import numpy as np
train_features['newer'] = (train_features.construction_year > 1998).map({True : 1, False : 0})
test_features['newer'] = (test_features.construction_year > 1998).map({True : 1, False : 0})

print("\ntrain_features unique nonumeric values")
print(train_features.select_dtypes(exclude=np.number).nunique())

y_train = train_labels.status_group


train_features unique nonumeric values
date_recorded              356
wpt_name                 37400
basin                        9
region                      21
lga                        125
ward                      2092
recorded_by                  1
extraction_type             18
extraction_type_group       13
extraction_type_class        7
management                  12
management_group             5
payment                      7
payment_type                 7
water_quality                8
quality_group                6
quantity                     5
quantity_group               5
source                      10
source_type                  7
source_class                 3
waterpoint_type              7
waterpoint_type_group        6
dtype: int64


In [0]:
pip install category_encoders

    100% |████████████████████████████████| 61kB 2.3MB/s 


In [0]:

from sklearn.preprocessing import OneHotEncoder
import category_encoders as ce
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

X_train = train_features.drop(['id', "date_recorded", "wpt_name", "lga", "ward"], axis = 1)
X_test  = test_features.drop(['id', "date_recorded", "wpt_name", "lga", "ward"], axis = 1)

cat_encoder = ce.OneHotEncoder(use_cat_names=True)

clf = RandomForestClassifier(class_weight = 'balanced',
                                            n_jobs = -1,
                                            n_estimators = 2000)

pipeline = make_pipeline(cat_encoder,
                         StandardScaler(),
                         clf)

pipe_out = pipeline.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:467: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)


In [0]:
print("RF Score:", pipeline.score(X_train, y_train))

submission = pd.read_csv('https://raw.githubusercontent.com/ssingh1187/DS-Unit-2-Sprint-4-Model-Validation/master/tanzania_kaggle_1.csv')


/usr/local/lib/python3.6/dist-packages/sklearn/pipeline.py:511: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


RF Score: 0.9929124579124579


In [0]:
temp = pd.DataFrame(pipeline.predict(X_test))

print(temp.shape)
print(temp[0].value_counts(normalize=True))
print(temp[0].value_counts())

submission['status_group'] = temp[0]
print(submission.status_group.value_counts())

submission.to_csv('tanzania_kaggle_02.csv', index=False)

/usr/local/lib/python3.6/dist-packages/sklearn/pipeline.py:331: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


(14358, 1)
functional                 0.570901
non functional             0.373032
functional needs repair    0.056066
Name: 0, dtype: float64
functional                 8197
non functional             5356
functional needs repair     805
Name: 0, dtype: int64
functional                 8197
non functional             5356
functional needs repair     805
Name: status_group, dtype: int64
